In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

      1.3
0       1
1       0
2       0
3       0
4       1
...   ...
1303    0
1304    0
1305    0
1306    0
1307    0

[1308 rows x 1 columns]


In [31]:
filename = "titanic_numerical_clean.csv"
X = pd.read_csv(filename, usecols=[0,25], sep=",", header=0)
Y = pd.read_csv(filename, usecols=[26], sep=",", header=0)
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.33)

print(len(trainX), 'training records and', len(testX), 'testing records')

def trainAndPredict(model):
    model.fit(trainX, trainY)
    predictions = model.predict(testX)
    mismatch = 0
    for estimate, real in zip(predictions, testY):
        if estimate != real:
            mismatch += 1
    return mismatch

876 training records and 432 testing records
